In [ ]:
"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

In [1]:
# Change dir to get SQL connector
import os
os.chdir("..")
from src.sqldb import SQLDatabase

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re

# Cleaning

In [3]:
# Data Objects
with SQLDatabase() as sql_db:
    event_table = sql_db.get_dataframe_from("event")
    deck_table = sql_db.get_dataframe_from("deck")
    deck_list_table = sql_db.get_dataframe_from("deckList")
    card_table = sql_db.get_dataframe_from("card")
    pilot_table = sql_db.get_dataframe_from("pilot")

In [4]:
# Clean Dates
event_table["date"] = pd.to_datetime(event_table["date"], dayfirst=True)

# Renaming ID Columns
event_table.rename(columns={"id": "eventId"}, inplace=True)
deck_table.rename(columns={"id": "deckId"}, inplace=True)
pilot_table.rename(columns={"id": "pilotId"}, inplace=True)

# Adding easier ID column to card_table
card_table["setNumber"] = card_table["setNumber"].str.zfill(3)
card_table["cardId"] = card_table["setNumber"] + card_table["setName"]

In [5]:
# Adding archetypes to deck_table
archetypes = {
    "aggro": ["aggro", "red deck wins", "weenie"],
    "control": ["control", "doom", "4cc", "yorion"],
    "mill": ["mill"],
    "ramp": ["ramp", "omnath"],
    "rogue": ["rogue"],
    "flash": ["flash"],
    "adventure": ["adventure"],
    "food": ["food"],
    "midrange": ["midrange"]
}

for archetype in archetypes:
    for placeholder in archetypes[archetype]:
        deck_table.loc[
            (deck_table.name.str.lower().str.contains(placeholder)),
            "archetype"
        ] = archetype

In [18]:
# Adding easy color groups to deck_table
categories = {
    "mono white": ["mono white", "mono-white", "white weenie", "weenie white", "WW"],
    "mono blue": ["mono blue", "mono-blue"],
    "mono black": ["mono black", "mono-black"],
    "mono red": ["mono red", "mono-red", "red deck wins"],
    "mono green": ["mono green", "mono-green", "monogreen"],
    "azorius": ["azorius", "uw", "wu"],
    "orzhov": ["orzhov", "wb", "bw"],
    "boros": ["boros", "wr", "rw", "winota"],
    "selesnya": ["selesnya", "wg", "gw"],
    "dimir": ["dimir", "ub"],
    "izzet": ["izzet", "ur"],
    "simic": ["simic", "gu"],
    "rakdos": ["rakdos", "br", "rb"],
    "golgari": ["golgari", "bg", "gb"],
    "gruul": ["gruul", "rg"],
    "bant": ["bant", "gwu", "guw", "ugw", "uwg", "wgu", "wug"],
    "esper": ["esper", "wub"],
    "grixis": ["grixis"],
    "jund": ["jund"],
    "naya": ["naya"],
    "jeskai": ["jeskai"],
    "sultai": ["sultai"],
    "mardu": ["mardu", "kroxa doom"],
    "temur": ["temur"],
    "abzan": ["abzan"],
    "4-color": ["omnath", "4c"],
    "colorless": ["forsaken monument"]
}

for category in categories:
    for placeholder in categories[category]:
        deck_table.loc[
            (deck_table.name.str.lower().str.contains(placeholder)),
            "category"
        ] = category

deck_table.loc[
    (deck_table.name.str.lower() == "gyruda"),
    "category"
] = "4-color"

In [19]:
# Replace all the abu codes with updated codes for basic lands 
# and Disenchant & Unsummon
abu_codes = {
    "092abu": "266eld",
    "170abu": "262eld",
    "130abu": "254eld",
    "196abu": "250eld",
    "249abu": "258eld",
    "065abu": "010znr",
    "271abu": "078m20"
}

other_codes = {
    "171cmd": "204m21", # Scavenging Ooze
    "356eld": "101eld", # Rankle, Master of Pranks
    "367eld": "147eld", # Torbran, Thane of Red Fell
    "335eld": "008eld", # Charming Prince
    "346eld": "054eld", # Midnight Clock
    "001unk": "", # Unknown card
    "003ivg": "174grn", # Goblin Electromancer
    "00410m": "004m10", # Baneslayer Angel
    "00514c": "013m21", # Containment Priest
    "021sha": "021shm", # Runed Halo
    "036str": "126eld", # Fling
    "0496th": "032m20", # Pacifism
    "05110m": "051m10", # Essence Scatter
    "05412m": "054m12", # Frost Breath
    "054m12 ": "054m12",
    "079sta": "103m21", # Grim Tutor
    "083urs": "096m21", # Duress
    "10113m": "101m13", # Murder
    "107dar": "241m21", # Tormod's Crypt
    "1136th": "159m21", # Shock
    "12410m": "124m10", # Act of Treason
    "1276th": "171m21", # Volcanic Geyser
    "144gui": "248rna", # Godless Shrine
    "162gui": "257grn", # Steam Vents
    "163gui": "259rna", # Stomping Ground
    "23215m": "245m15", # Radiant Fountain
    "237urs": "063m21", # Rewind
    "309thb": "076thb", # Thryx, the Sudden Storm
    "328iko": "162iko", # Kogla, the Titan Ape
    "334eld": "001eld", # Acclaimed Contender
    "364znr": "215znr"  # Turntimber Symbiosis // Turntimber, Serpentine...
}

card_table["cardId"].replace(to_replace=abu_codes, inplace=True)
deck_list_table["cardId"].replace(to_replace=abu_codes, inplace=True)

deck_list_table["cardId"].replace(to_replace=other_codes, inplace=True)

In [33]:
unmatched = set(deck_list_table.loc[~(deck_list_table.cardId.isin(card_table.cardId))]["cardId"])

def find_urls_with_unmatched(unmatched):
    unmatched_deck_ids = list(deck_list_table.loc[deck_list_table.cardId == unmatched]["deckId"])
    return set(deck_table.loc[deck_table.deckId.isin(unmatched_deck_ids)]["deckUrl"])

def print_page_source(unmatched):
    unmatched_url = find_urls_with_unmatched(unmatched).pop()
    print(unmatched_url)
    html = requests.get("https://www.mtgtop8.com/event" + unmatched_url)
    return BeautifulSoup(html.text, features="lxml")

In [21]:
events_and_decks = pd.merge(
    event_table, 
    deck_table, 
    on="eventId",
    suffixes=["_event", "_deck"]
).drop_duplicates()

events_decks_pilots = pd.merge(
    events_and_decks,
    pilot_table,
    on="pilotId",
    suffixes=[None, "_pilot"]
).drop_duplicates()

full_table_no_cards = pd.merge(
    events_decks_pilots,
    deck_list_table,
    on="deckId",
    suffixes=[None, "_decklist"]
).drop_duplicates()

full_table = pd.merge(
    full_table_no_cards,
    card_table,
    on="cardId",
    suffixes=[None, "_cards"]
)[[
    "eventId",
    "name_event",
    "date",
    "deckId",
    "pilotId",
    "name_deck",
    "firstName",
    "lastName",
    "cardId",
    "name",
    "count",
    "color",
    "slot",
    "archetype",
    "category"
]].drop_duplicates().sort_values(by=["eventId", "deckId", "slot"]).reset_index(drop=True)

In [22]:
full_table.drop_duplicates("deckId").groupby("archetype")["deckId"].count().sort_values(ascending=False)

archetype
aggro        1021
control       523
ramp          272
mill          132
adventure     107
rogue          80
midrange        5
food            4
flash           3
Name: deckId, dtype: int64

In [23]:
full_table.loc[
    full_table["date"] >= "2020-09-25"
].drop_duplicates("deckId").groupby(["archetype"])["deckId"].count().sort_values(ascending=False)

archetype
aggro        912
control      425
ramp         201
mill         131
rogue         80
adventure     78
midrange       5
food           4
flash          1
Name: deckId, dtype: int64

In [24]:
full_table.loc[
    full_table["date"] < "2020-09-28"
].drop_duplicates(["deckId"]).groupby("archetype")["deckId"].count().sort_values(ascending=False)

archetype
aggro        130
ramp         126
control      123
adventure     46
flash          2
rogue          1
mill           1
midrange       1
Name: deckId, dtype: int64

In [25]:
full_table.drop_duplicates("deckId").groupby("category")["deckId"].count().sort_values(ascending=False)

category
gruul         426
dimir         277
4-color       204
mono green    185
rakdos        139
esper         134
mono red      130
temur         130
sultai        105
simic          88
izzet          61
azorius        56
selesnya       51
golgari        48
boros          46
mono black     24
mono white     19
jund           18
abzan          14
orzhov         12
grixis         12
naya           10
jeskai          9
bant            9
mardu           5
colorless       4
mono blue       4
Name: deckId, dtype: int64

In [26]:
full_table.loc[
    full_table["date"] >= "2020-09-28"
].drop_duplicates(["deckId"]).groupby("category")["deckId"].count().sort_values(ascending=False)

category
gruul         402
dimir         263
mono green    146
esper         131
rakdos        124
temur         111
mono red       97
simic          84
4-color        75
azorius        52
selesnya       48
golgari        47
boros          36
izzet          22
mono white     19
mono black     19
jund           17
abzan          14
grixis         12
sultai         11
naya           10
jeskai          9
orzhov          9
bant            9
mono blue       4
colorless       4
mardu           4
Name: deckId, dtype: int64

In [27]:
full_table.loc[
    full_table["date"] < "2020-09-28"
].drop_duplicates(["deckId"]).groupby("category")["deckId"].count().sort_values(ascending=False)

category
4-color       129
sultai         94
izzet          39
mono green     39
mono red       33
gruul          24
temur          19
rakdos         15
dimir          14
boros          10
mono black      5
simic           4
azorius         4
orzhov          3
selesnya        3
esper           3
jund            1
golgari         1
mardu           1
Name: deckId, dtype: int64

In [28]:
full_table.loc[
    full_table["archetype"] == "flash"
]["deckId"].value_counts()

1101    32
1038    30
21      26
Name: deckId, dtype: int64

In [29]:
full_table.loc[full_table.deckId == 2186]

,eventId,name_event,date,deckId,pilotId,name_deck,firstName,lastName,cardId,name,count,color,slot,archetype,category
58939,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,250eld,Plains,22,W,md,NaN,NaN
58940,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,036m21,Selfless Savior,4,W,md,NaN,NaN
58941,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,001thb,Alseid of Life's Bounty,4,W,md,NaN,NaN
58942,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,009thb,"Daxos, Blessed by the Sun",3,W,md,NaN,NaN
58943,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,024znr,Luminarch Aspirant,1,W,md,NaN,NaN
58944,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,018eld,Hushbringer,1,W,md,NaN,NaN
58945,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,011eld,Faerie Guidemother // Gift of the Fae,4,W,md,NaN,NaN
58946,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,219eld,Gingerbrute,4,,md,NaN,NaN
58947,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,010ktk,Feat of Resistance,4,W,md,NaN,NaN
58948,415,Gentry ! @ The Gentry Magic League,2020-12-08,2186,1369,WW All That Glitters,Tom,De Wael,002eld,All That Glitters,4,W,md,NaN,NaN


In [32]:
full_table.loc[
    full_table.category.isna()
][["deckId", "name_deck", "category"]]

,deckId,name_deck,category
9631,360,Other - Aggro,NaN
9632,360,Other - Aggro,NaN
9633,360,Other - Aggro,NaN
9634,360,Other - Aggro,NaN
9635,360,Other - Aggro,NaN
...,...,...,...
58952,2186,WW All That Glitters,NaN
58953,2186,WW All That Glitters,NaN
58954,2186,WW All That Glitters,NaN
58955,2186,WW All That Glitters,NaN


In [31]:
full_table.loc[
    ~(full_table.name.isin(["Forest", "Swamp", "Mountain", "Island", "Plains"]))
].groupby("name")["count"].sum().sort_values(ascending=False).head(30)

name
Fabled Passage                                            5961
Bonecrusher Giant // Stomp                                3918
Lovestruck Beast // Heart's Desire                        3110
Mystical Dispute                                          2997
Cragcrown Pathway // Timbercrown Pathway                  2871
Kazandu Mammoth // Kazandu Valley                         2598
Edgewall Innkeeper                                        2435
Shatterskull Smashing // Shatterskull, the Hammer Pass    2384
Scavenging Ooze                                           2258
Negate                                                    2096
Heartless Act                                             2043
Clearwater Pathway // Murkwater Pathway                   1998
Embercleave                                               1886
Bloodchief's Thirst                                       1837
The Great Henge                                           1657
Shark Typhoon                                     

In [193]:
full_table.groupby("name_deck").count().sort_values(by="count", ascending=False).head(30)

,eventId,name_event,date,deckId,pilotId,firstName,lastName,cardId,name,count,color,slot,archetype,category
name_deck,,,,,,,,,,,,,,
Gruul Aggro,8980,8980,8980,8980,8980,8980,8980,8980,8980,8980,8980,8980,8980,8980
Omnath Ramp,5374,5374,5374,5374,5374,5374,5374,5374,5374,5374,5374,5374,5374,5374
Mono Green Aggro,4166,4166,4166,4166,4166,4166,4166,4166,4166,4166,4166,4166,4166,4166
Esper DOOM Yorion,4072,4072,4072,4072,4072,4072,4072,4072,4072,4072,4072,4072,4072,4072
Rakdos Aggro,3148,3148,3148,3148,3148,3148,3148,3148,3148,3148,3148,3148,3148,3148
Sultai Control,3091,3091,3091,3091,3091,3091,3091,3091,3091,3091,3091,3091,3091,3091
Dimir Mill,2806,2806,2806,2806,2806,2806,2806,2806,2806,2806,2806,2806,2806,2806
Red Deck Wins,2422,2422,2422,2422,2422,2422,2422,2422,2422,2422,2422,2422,2422,2422
Dimir Rogue,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014


### QUESTION 1: 

How has color dominance changed over time?